In [2]:
import os #, re, datetime
import pandas as pd
import numpy as np

os.chdir(path)
path = 'C:/Users/bgeorgiev/Documents/Python Scripts/Automate/'


infile = 'TFS UK Issue3-31-19.xlsx'
fields = 'TFS UK Issue3-31-19.xlsx'


file  = pd.read_excel(path + infile, sheet_name = 'Data' ,index_col=0)

datatypes = pd.read_excel(path + infile,sheet_name  = 'Field Size', index_col=0)
datatypes = datatypes.fillna(0)

In [3]:
#datatypes.head()

In [4]:
datasize = datatypes[datatypes['Data Type']!= 'DateTime']
datasize = datasize[['Physical Name','Data Type', 'Size', 'Precision','Scale']]
#datasize['Length'] = np.where(datasize['Data Type'].str.lower() == 'integer', datasize['Precision'], np.where(datasize['Data Type'].str.lower() == 'char', datasize['Size'], datasize['Precision'] -  datasize['Scale'] -1))
datasize['Length'] = np.where(datasize['Data Type'].str.lower() == 'integer', datasize['Precision'].replace(0,datasize['Size']), np.where((datasize['Data Type'].str.lower() == 'char') | (datasize['Data Type'].str.lower() == 'string'), datasize['Size'], datasize['Precision'] -  datasize['Scale']))
datasize = datasize.loc[:,['Physical Name', 'Scale','Length']]
datasize = datasize.set_index('Physical Name')
datasize.index = datasize.index.str.strip()
datasize.head()
datasize.sort_index(axis=0, ascending=True, inplace=True)
datasize.head()

,Scale,Length
Physical Name,,
MPool_ID,0.0,10.0
TRUST_ID,0.0,10.0
PCD_SALE_ID,0.0,10.0
Agreement_Number,0.0,12.0
Accnt_Tot_Ass_Displ_Loss_Cum_M,2.0,13.0


,Scale,Length
Physical Name,,
AM_SCHED_Arrears_Included,4.0,14.0
AM_SCHED_Non_Int_Earning_Assets,4.0,14.0
AM_SCHED_Overpayments_Included,4.0,14.0
AM_SCHED_Suspended_Included,4.0,14.0
Accnt_Tot_Ass_Displ_Loss_Cum_M,2.0,13.0


In [5]:
#print(sorted(set(datasize['Physical Name'])))

In [6]:
filemax =  pd.DataFrame(file.max())
filemin =  pd.DataFrame(file.min())
fileminmax = pd.concat([filemin, filemax], axis = 1) #, ignore_index = 'True') 
fileminmax.columns = ['Min', 'Max']
fileminmax.index.name = 'Physical Name'
fileminmax.index = fileminmax.index.str.strip()
fileminmax.sort_index(axis=0, ascending=True, inplace=True)
fileminmax.head()

,Min,Max
Physical Name,,
AM_SCHED_Arrears_Included,0,252433
AM_SCHED_Non_Int_Earning_Assets,0,0
AM_SCHED_Overpayments_Included,0,0
AM_SCHED_Suspended_Included,0,78629.1
Accnt_Tot_Ass_Displ_Loss_Cum_M,-32726.8,3.15807e+06


In [7]:
fileminmax.info()
fileminmax.head()

<class 'pandas.core.frame.DataFrame'>
Index: 124 entries, AM_SCHED_Arrears_Included to WA_YIELD
Data columns (total 2 columns):
Min    124 non-null object
Max    124 non-null object
dtypes: object(2)
memory usage: 2.9+ KB


,Min,Max
Physical Name,,
AM_SCHED_Arrears_Included,0,252433
AM_SCHED_Non_Int_Earning_Assets,0,0
AM_SCHED_Overpayments_Included,0,0
AM_SCHED_Suspended_Included,0,78629.1
Accnt_Tot_Ass_Displ_Loss_Cum_M,-32726.8,3.15807e+06


In [8]:
datasize.info()
datasize.head()

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, AM_SCHED_Arrears_Included to WA_YIELD
Data columns (total 2 columns):
Scale     135 non-null float64
Length    135 non-null float64
dtypes: float64(2)
memory usage: 3.2+ KB


,Scale,Length
Physical Name,,
AM_SCHED_Arrears_Included,4.0,14.0
AM_SCHED_Non_Int_Earning_Assets,4.0,14.0
AM_SCHED_Overpayments_Included,4.0,14.0
AM_SCHED_Suspended_Included,4.0,14.0
Accnt_Tot_Ass_Displ_Loss_Cum_M,2.0,13.0


In [9]:
#fileminmax.index
#datasize.index

In [10]:
fileminmax.info()
fileminmax.head()

<class 'pandas.core.frame.DataFrame'>
Index: 124 entries, AM_SCHED_Arrears_Included to WA_YIELD
Data columns (total 2 columns):
Min    124 non-null object
Max    124 non-null object
dtypes: object(2)
memory usage: 2.9+ KB


,Min,Max
Physical Name,,
AM_SCHED_Arrears_Included,0,252433
AM_SCHED_Non_Int_Earning_Assets,0,0
AM_SCHED_Overpayments_Included,0,0
AM_SCHED_Suspended_Included,0,78629.1
Accnt_Tot_Ass_Displ_Loss_Cum_M,-32726.8,3.15807e+06


In [11]:
idxd = datasize.index
idxf = fileminmax.index
idxd.difference(idxf)

Index(['Agreement_Number', 'Asset_Status_Month', 'Customer_Number', 'MPool_ID',
       'PCD_SALE_ID', 'Rentals_In_Arrears_Month_Group', 'Repurchased_Pool',
       'S_AcctStatus', 'S_AcctStatusPreCSEOM', 'S_InvEndPrinBal',
       'Sold_Pool_Month', 'Status_Month', 'Sub_Status_Month'],
      dtype='object', name='Physical Name')

In [12]:
new = pd.merge(datasize,fileminmax,how='left', left_index=True, right_index=True)


new['Min'] = new['Min'].fillna(0)
new['Max'] = new['Max'].fillna(0)

new['MinInt'] = pd.to_numeric(new['Min'], errors='coerce')
new['MinInt'] = new['MinInt'].fillna(0)
#new = new.dropna(subset=['MinInt'])
new['MinInt'] = abs(new['MinInt'].astype('int'))


new['MaxInt'] = pd.to_numeric(new['Max'], errors='coerce')
new['MaxInt'] = new['MaxInt'].fillna(0)
#new = new.dropna(subset=['MinInt'])
new['MaxInt'] = abs(new['MaxInt'].astype('int'))

In [13]:
new['MaxBreak'] = np.where(new['Length'] - new['MaxInt'].map(str).apply(len) < 0 , 'Y','')
new['MinBreak'] = np.where(new['Length'] - new['MinInt'].map(str).apply(len) < 0 , 'Y','')

issues = new[(new['MaxBreak'] == 'Y')| (new['MinBreak'] == 'Y')]
print(issues)

                Scale  Length Min      Max  MinInt  MaxInt MaxBreak MinBreak
Physical Name                                                               
PctBalloonPmts   15.0     3.0   0  2359.36       0    2359        Y         
